# Pricing Option Surface

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.interpolate import griddata
import os

# local
from LSM import MonteCarloOptionPricing
from DC import DynamicChebyshev

In [9]:
# Initialize instance from class
mc_option_pricing = MonteCarloOptionPricing(r=0.06, S0=36., K=40., T=1., σ=0.2, λ=0.4, dim=10_000, n=50, seed=150_000, use_AV = True)

# Generate 'dim' no. of stock price process paths
simulated_paths = mc_option_pricing.GeometricBrownianMotion_vec()

# plot stock price process
mc_option_pricing.american_option_LSM(poly_degree = 3, otype= 'put')

(4.440780819406584,
 array([1.  , 0.04, 0.74, ..., 0.3 , 0.72, 0.1 ]),
 array([0.        , 6.11946468, 6.29024145, ..., 6.29572025, 4.79791791,
        7.26447831]))

In [10]:
def price_options_for_various_parameters():
    # Parameters
    r = 0.06  # risk-free rate
    K = 40.   # strike price
    dim = 100000  # number of simulation paths
    n = 50  # number of exercise rights per year
    seed = 15001  # random seed for reproducibility
    use_AV = True  # antithetic variates
    poly_degree = 3  # polynomial degree for LSM regression

    # Different characteristics
    S0_values = [36, 38, 40, 42, 44]
    sigma_values = [0.20, 0.40]
    T_values = [1, 2]

    # Store results
    results = []

    # Iterate over each combination of S, sigma, and T
    for S0 in S0_values:
        for sigma in sigma_values:
            for T in T_values:
                # Initialize the Monte Carlo pricer
                mc_option_pricing = MonteCarloOptionPricing(r, S0, K, T, sigma, dim, n, seed, use_AV)
                # Simulate the asset paths
                mc_option_pricing.GeometricBrownianMotion_vec()
                # Price the option using LSM
                option_price = mc_option_pricing.american_option_LSM(poly_degree, otype='put')
                # Store the results
                results.append((S0, sigma, T, option_price))
                # Print individual values without formatting the entire tuple
                print(f'S: {S0}, Sigma: {sigma}, T: {T}, American Put Option Price: {option_price[0]:.4f}')

    return results

# Call the function to price the options
option_prices = price_options_for_various_parameters()

S: 36, Sigma: 0.2, T: 1, American Put Option Price: 4.4720
S: 36, Sigma: 0.2, T: 2, American Put Option Price: 4.8259
S: 36, Sigma: 0.4, T: 1, American Put Option Price: 7.0751
S: 36, Sigma: 0.4, T: 2, American Put Option Price: 8.4595
S: 38, Sigma: 0.2, T: 1, American Put Option Price: 3.2400
S: 38, Sigma: 0.2, T: 2, American Put Option Price: 3.7267
S: 38, Sigma: 0.4, T: 1, American Put Option Price: 6.1222
S: 38, Sigma: 0.4, T: 2, American Put Option Price: 7.6295
S: 40, Sigma: 0.2, T: 1, American Put Option Price: 2.3049
S: 40, Sigma: 0.2, T: 2, American Put Option Price: 2.8727
S: 40, Sigma: 0.4, T: 1, American Put Option Price: 5.2752
S: 40, Sigma: 0.4, T: 2, American Put Option Price: 6.8784
S: 42, Sigma: 0.2, T: 1, American Put Option Price: 1.6036
S: 42, Sigma: 0.2, T: 2, American Put Option Price: 2.1992
S: 42, Sigma: 0.4, T: 1, American Put Option Price: 4.5470
S: 42, Sigma: 0.4, T: 2, American Put Option Price: 6.1980
S: 44, Sigma: 0.2, T: 1, American Put Option Price: 1.09

In [11]:
def price_american_options():
    r = 0.03  # risk-free rate
    S0 = 100  # current stock price
    σ = 0.25  # volatility
    dim = 1_000  # number of simulation paths
    n = 504  # number of exercise rights per year
    seed = 42  # random seed for reproducibility
    poly_degree = 3  # polynomial degree for LSM regression
    # Define maturities and strikes
    maturities = np.array([1/12, 2/12, 3/12, 6/12, 9/12, 1, 15/12, 18/12, 2, 30/12, 3, 4])
    strikes = np.arange(0.8 * S0, 1.21 * S0, 0.05 * S0)

    # Lists to store results
    prices = []
    maturity_list = []
    strike_list = []

    # Iterate over each maturity and strike combination
    for T in maturities:
        for K in strikes:
            # Initialize the Monte Carlo pricer
            mcpricer = MonteCarloOptionPricing(r, S0, K, T, σ, dim, n, seed)
            # Simulate the asset paths
            mcpricer.GeometricBrownianMotion_vec()
            # Price the option using LSM
            option_price = mcpricer.american_option_LSM(poly_degree, otype='put')

            # Store the results
            prices.append(option_price)
            maturity_list.append(T)
            strike_list.append(K)

    return prices, maturity_list, strike_list

# Call the function to get the results
prices, maturities, strikes = price_american_options()

# Convert lists to numpy arrays for processing
maturities_np = np.array(maturities)
strikes_np = np.array(strikes)
prices_np = np.array(prices)

# Creating a grid for maturities and strikes
maturity_grid, strike_grid = np.meshgrid(np.linspace(maturities_np.min(), maturities_np.max(), 100),
                                         np.linspace(strikes_np.min(), strikes_np.max(), 100))

# Interpolating the prices onto the grid
price_grid = griddata((maturities_np, strikes_np), prices_np, (maturity_grid, strike_grid), method='cubic')

# Plotting the surface
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
surface = ax.plot_surface(maturity_grid, strike_grid, price_grid, cmap='viridis', edgecolor='none')
ax.set_xlabel('Maturity (Years)')
ax.set_ylabel('Strike Price')
ax.set_zlabel('Option Price')
plt.title('American Put Option Prices Surface (LSM)')

plt.tight_layout()

if not os.path.exists("../Results/plots"):
    os.makedirs("../Results/plots")
    
plt.savefig("../Results/plots/OptionsSurfaceLSM.pdf", bbox_inches='tight')

# Display the plot
plt.show()

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (108, 2) + inhomogeneous part.

In [ ]:
# Define a range of maturities and strikes
S0 = 100
σ = 0.25
r = 0.03
dim = 1000
n_chebyshev_pol = 100
n_timesteps = 50
seed = 150000

maturities_range = np.array([1/12, 2/12, 3/12, 6/12, 9/12, 1, 15/12, 18/12, 2, 30/12, 3, 4])
strikes_range = np.linspace(0.8 * S0, 1.2 * S0, 10)

# Calculate option prices for each maturity and strike
option_prices = []
maturity_list = []
strike_list = []

for T in maturities_range:
    for K in strikes_range:
        price = chebyshev_option_price(S0, K, T, σ, r, dim, n_chebyshev_pol, n_timesteps, seed)
        option_prices.append(price)
        maturity_list.append(T)
        strike_list.append(K)

# Prepare data for plotting
maturities_np = np.array(maturity_list)
strikes_np = np.array(strike_list)
prices_np = np.array(option_prices)

# Creating a grid for maturities and strikes
maturity_grid, strike_grid = np.meshgrid(np.linspace(maturities_np.min(), maturities_np.max(), 100),
                                         np.linspace(strikes_np.min(), strikes_np.max(), 100))

# Interpolating the prices onto the grid
price_grid = griddata((maturities_np, strikes_np), prices_np, (maturity_grid, strike_grid), method='cubic')

# Plotting the surface
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(maturity_grid, strike_grid, price_grid, cmap='viridis', edgecolor='none')
ax.set_xlabel('Maturity (Years)')
ax.set_ylabel('Strike Price')
ax.set_zlabel('Option Price')
plt.title('Option Prices using Chebyshev Method')

plt.tight_layout()

if not os.path.exists("../Results/plots"):
    os.makedirs("../Results/plots")
    
plt.savefig("../Results/plots/ChebyshevMethodSurface.pdf", bbox_inches='tight')

# Show the plot
plt.show()